In [151]:
import pandas as pd
import numpy as np
import random

import pyltr
from sklearn.model_selection import train_test_split

train = pd.read_csv("data/training_set_VU_DM_2014.csv")

In [152]:
train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [153]:
train.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

In [154]:
len(train.loc[train.booking_bool == 1]) / len(train) * 100

2.7910511305481442

In [155]:
len(train.loc[(train.click_bool == 1) & (train.booking_bool == 0)]) / len(train) * 100

1.6838071236240628

In [156]:
len(train.loc[train.click_bool == 0]) / len(train) * 100

95.5251417458278

In [157]:
# Generate target label
train["relevance"] = train["click_bool"] + train["booking_bool"]
train["relevance"] = train["relevance"].map(lambda x: 5 if x > 1 else x)

In [158]:
# Delete click, booking and gross_booking
del(train["click_bool"])
del(train["booking_bool"])
del(train["gross_bookings_usd"])

train = train[["srch_id", "site_id", "prop_country_id", "prop_starrating", "prop_review_score", "prop_brand_bool",
              "prop_location_score1", "prop_location_score2", "price_usd", "promotion_flag", "srch_destination_id",
              "srch_length_of_stay", "srch_adults_count", "srch_children_count", "srch_room_count", "relevance"]]
train = train.fillna(-1)

train.head()

,srch_id,site_id,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_adults_count,srch_children_count,srch_room_count,relevance
0,1,12,219,3,3.5,1,2.83,0.0438,104.77,0,23246,1,4,0,1,0
1,1,12,219,4,4.0,1,2.20,0.0149,170.74,0,23246,1,4,0,1,0
2,1,12,219,3,4.5,1,2.20,0.0245,179.80,0,23246,1,4,0,1,0
3,1,12,219,2,4.0,1,2.83,0.0125,602.77,0,23246,1,4,0,1,0
4,1,12,219,4,3.5,1,2.64,0.1241,143.58,0,23246,1,4,0,1,0


In [220]:
# Downsample a series so that there are as many non-clicked items as clicked and booked
def downsample_series(s):
    s1 = s.where(lambda e: e["relevance"] > 0).dropna()
    s2 = s.where(lambda e: e["relevance"] == 0).dropna()
    s2 = s2.sample(frac=min(1, len(s1)/len(s2)))
    return s1.append(s2).sort_index()

# Downsample a DF so that there are as many non-clicked items as clicked and booked
def downsample(df):
    return df.groupby("srch_id").apply(downsample_series).reset_index(drop=True)

# Generate train and test set from original train set
srch_ids = train["srch_id"].unique()

srch_ids_train, srch_ids_test = train_test_split(srch_ids, test_size=0.2, random_state = 42)

tr_train = train.loc[train.srch_id.isin(srch_ids_train)]
tr_test = train.loc[train.srch_id.isin(srch_ids_test)]

tr_train = downsample(tr_train)

TX = tr_train.drop("srch_id", 1).drop("relevance", 1)
Ty = tr_train["relevance"]
Tqids = tr_train["srch_id"]

EX = tr_test.drop("srch_id", 1).drop("relevance", 1)
Ey = tr_test["relevance"]
Eqids = tr_test["srch_id"]

len(tr_train) / len(train)

ZeroDivisionError: division by zero

In [219]:
max(1,2)

2

In [215]:
def downsample_series(s):
    s1 = s.where(lambda e: e["relevance"] > 0).dropna()
    s2 = s.where(lambda e: e["relevance"] == 0).dropna()
    s2 = s2.sample(frac=(len(s1)/len(s2)))
    return s1.append(s2).sort_index()

x = train.head(n=500)
x.groupby("srch_id").apply(downsample_series)[["srch_id", "relevance"]].reset_index(drop=True)


,srch_id,relevance
0,1.0,5.0
1,1.0,0.0
2,4.0,0.0
3,4.0,1.0
4,6.0,5.0
5,6.0,0.0
6,8.0,5.0
7,8.0,0.0
8,11.0,0.0
9,11.0,1.0


In [118]:
# Train model
metric = pyltr.metrics.NDCG(k=38)
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=100,
    learning_rate=0.02,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)
model.fit(TX, Ty, Tqids)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.3307       0.1016      378.34m                                         
    2       0.3955       0.0656      355.96m                                         
    3       0.4072       0.0106      346.05m                                         
    4       0.4090       0.0022      341.55m                                         
    5       0.4103       0.0017      340.51m                                         
    6       0.4179       0.0069      336.99m                                         


KeyboardInterrupt: 

In [ ]:
# Test model
Epred = model.predict(EX)
print 'Random ranking:', metric.calc_mean_random(Eqids, Ey)
print 'Our model:', metric.calc_mean(Eqids, Ey, Epred)

In [ ]:
ds = train.groupby("srch_id")
ds.apply(lambda x: )

In [150]:
x = pd.Series([0, 0, 1, 0, 0, 0, 0, 5, 0, 0])
y1 = x.where(lambda e: e > 0).dropna()
y2 = x.where(lambda e: e == 0).dropna()
y2 = y2.sample(frac=(len(y1)/len(y2)))

y = y1.append(y2).sort_index()
y

2    1.0
3    0.0
7    5.0
8    0.0
dtype: float64

In [160]:
pd.Series([0, 0, 1, 0, 0, 0, 0, 5, 0, 0]).sample(frac=0)

Series([], dtype: int64)

In [163]:
tr_train.groupby("srch_id").apply(downsample_series)

ValueError: a must be greater than 0

ZeroDivisionError: division by zero

In [203]:
x = train.head(n = 100)
x = downsample(x)
x[["srch_id", "relevance"]]

,srch_id,relevance
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
5,1.0,0.0
6,1.0,0.0
7,1.0,0.0
8,1.0,0.0
9,1.0,0.0
